## Classifying College as Private or Not?

### Importing PySpark

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.1.1-bin-hadoop3.2')

In [2]:
from pyspark.sql import SparkSession

In [3]:
#Initializing Spark Session
spark = SparkSession.builder.appName('dtrf').getOrCreate()

### Importing Data

In [4]:
#Reading Data
data = spark.read.csv('College.csv',inferSchema=True,header=True)

In [5]:
data.show(5)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|
|      Adrian College|    Yes|1428|  1097|   336|       22|       50|       1036|         99|  

## Exploratory Data Analysis

In [6]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [7]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [8]:
#Statistical description of data
data.describe().show()

+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
|summary|              School|Private|              Apps|            Accept|          Enroll|         Top10perc|         Top25perc|      F_Undergrad|      P_Undergrad|          Outstate|        Room_Board|             Books|          Personal|               PhD|          Terminal|         S_F_Ratio|       perc_alumni|          Expend|         Grad_Rate|
+-------+--------------------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------

### Shape of Dataset

In [9]:
print((data.count(), len(data.columns)))

(777, 19)


In [10]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

### Missing Values

In [11]:
from pyspark.sql.functions import isnan, when, count, col

In [12]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|     0|      0|   0|     0|     0|        0|        0|          0|          0|       0|         0|    0|       0|  0|       0|        0|          0|     0|        0|
+------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+



## Data Preprocessing

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
assembler = VectorAssembler(inputCols=['Apps','Accept','Enroll','Top10perc','Top25perc','F_Undergrad','P_Undergrad','Outstate',
                                     'Room_Board','Books','Personal','PhD','Terminal','S_F_Ratio','perc_alumni','Expend','Grad_Rate'],
                           outputCol='features')

#inputCols takes all the independent features
#outputCol helps to create a dense vector all the inputCols given with name can be anything. Here given 'features'

In [15]:
#Transforming whole data by Vector Assembler
output = assembler.transform(data)

In [16]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[1660.0,1232.0,72...|
|[2186.0,1924.0,51...|
|[1428.0,1097.0,33...|
|[417.0,349.0,137....|
|[193.0,146.0,55.0...|
|[587.0,479.0,158....|
|[353.0,340.0,103....|
|[1899.0,1720.0,48...|
|[1038.0,839.0,227...|
|[582.0,498.0,172....|
|[1732.0,1425.0,47...|
|[2652.0,1900.0,48...|
|[1179.0,780.0,290...|
|[1267.0,1080.0,38...|
|[494.0,313.0,157....|
|[1420.0,1093.0,22...|
|[4302.0,992.0,418...|
|[1216.0,908.0,423...|
|[1130.0,704.0,322...|
|[3540.0,2001.0,10...|
+--------------------+
only showing top 20 rows



In [17]:
output.head(1)

#Features here being a dense vector of all the inputCols given.

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60, features=DenseVector([1660.0, 1232.0, 721.0, 23.0, 52.0, 2885.0, 537.0, 7440.0, 3300.0, 450.0, 2200.0, 70.0, 78.0, 18.1, 12.0, 7041.0, 60.0]))]

### Transforming Label

In [18]:
from pyspark.ml.feature import StringIndexer

In [19]:
indexer = StringIndexer(inputCol='Private',outputCol='PrivateIndex')

In [20]:
output_fixed = indexer.fit(output).transform(output)

In [21]:
#Combining independent features and dependent features into a dataframe
final_data = output_fixed.select(['features','PrivateIndex'])

#Spark accepts thing kind of format

In [22]:
final_data.show(5)

+--------------------+------------+
|            features|PrivateIndex|
+--------------------+------------+
|[1660.0,1232.0,72...|         0.0|
|[2186.0,1924.0,51...|         0.0|
|[1428.0,1097.0,33...|         0.0|
|[417.0,349.0,137....|         0.0|
|[193.0,146.0,55.0...|         0.0|
+--------------------+------------+
only showing top 5 rows



## Model Building

In [23]:
#Splitting data into training and testing
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [24]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

In [25]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex')
gbtc = GBTClassifier(labelCol='PrivateIndex')
rfc = RandomForestClassifier(labelCol='PrivateIndex')

In [26]:
dtc_model = dtc.fit(train_data)
gbtc_model = gbtc.fit(train_data)
rfc_model = rfc.fit(train_data)

In [27]:
dtc_preds = dtc_model.transform(test_data)
gbtc_preds = gbtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)

## Evaluation Metrics

In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [29]:
my_binary_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='PrivateIndex')

### AUC Score Comparison

In [30]:
print('DTC')
print(my_binary_eval.evaluate(dtc_preds))
print('GBTC')
print(my_binary_eval.evaluate(gbtc_preds))
print('RFC')
print(my_binary_eval.evaluate(rfc_preds))

DTC
0.8574464153732445
GBTC
0.8725979305247598
RFC
0.8938470066518848


### Accuracy

In [31]:
#Helps to calculate 'precision','recall','f1','accuracy'
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
#Calculate accuracy
my_multiclass_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex',metricName='accuracy')

In [37]:
print('DTC')
print(my_multiclass_eval.evaluate(dtc_preds))
print('GBTC')
print(my_multiclass_eval.evaluate(gbtc_preds))
print('RFC')
print(my_multiclass_eval.evaluate(rfc_preds))

DTC
0.9
GBTC
0.908695652173913
RFC
0.9260869565217391
